In [8]:
import tensorflow as tf
import sys
import matplotlib.pyplot as plt
import tensorflow_hub as tfhb

In [9]:
print(tf.__version__)
print(sys.version)
print(tf.config.list_physical_devices("GPU"))

2.10.0
3.10.0 | packaged by conda-forge | (default, Nov 10 2021, 13:20:59) [MSC v.1916 64 bit (AMD64)]
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [10]:
train_data_dir = r"D:\Machine Learning\Sample Project1\Notes Notebooks\Working_Data\10_food_classes_10_percent\10_food_classes_10_percent\train"
test_data_dir = r"D:\Machine Learning\Sample Project1\Notes Notebooks\Working_Data\10_food_classes_10_percent\10_food_classes_10_percent\test"

In [11]:
efficient_net_b0_fv_link = "https://tfhub.dev/google/efficientnet/b0/feature-vector/1"

In [12]:
train_ds_mn_224 , valid_ds_mn_224 = tf.keras.utils.image_dataset_from_directory(train_data_dir,
                                                                                label_mode="categorical",
                                                                                validation_split=0.15,
                                                                                shuffle=True,
                                                                                seed=42,
                                                                                subset="both",
                                                                                image_size=(224,224),
                                                                                batch_size=32)

test_ds_mn_224 = tf.keras.utils.image_dataset_from_directory(test_data_dir,
                                                             label_mode="categorical",
                                                             shuffle=True,
                                                             image_size=(224,224),
                                                             batch_size=32)

Found 750 files belonging to 10 classes.
Using 638 files for training.
Using 112 files for validation.
Found 2500 files belonging to 10 classes.


In [6]:
efficient_net_b0_fv_base_model = tfhb.KerasLayer(efficient_net_b0_fv_link,
                                                 trainable=False)

In [7]:
input_layer = tf.keras.Input(shape=(224,224,3))

x = tf.keras.layers.Rescaling(1.0/225.0)(input_layer)
x = efficient_net_b0_fv_base_model(x)

output_layer = tf.keras.layers.Dense(10,activation="softmax")(x)

In [8]:
efficient_net_b0_fv_main_model = tf.keras.Model(input_layer,output_layer)

In [9]:
efficient_net_b0_fv_main_model.compile(loss="categorical_crossentropy",
                                       optimizer=tf.keras.optimizers.Adam(),
                                       metrics=["accuracy"])

In [10]:
efficient_net_b0_fv_main_model.fit(train_ds_mn_224,
                                   epochs=10,
                                   steps_per_epoch=len(train_ds_mn_224),
                                   validation_data=valid_ds_mn_224,
                                   validation_steps=len(valid_ds_mn_224))

Epoch 1/10
20/20 [==============================] - 8s 135ms/step - loss: 1.9213 - accuracy: 0.4154 - val_loss: 1.5524 - val_accuracy: 0.6071
Epoch 2/10
20/20 [==============================] - 1s 65ms/step - loss: 1.1449 - accuracy: 0.7508 - val_loss: 1.1638 - val_accuracy: 0.6696
Epoch 3/10
20/20 [==============================] - 1s 66ms/step - loss: 0.8200 - accuracy: 0.8339 - val_loss: 0.9916 - val_accuracy: 0.7143
Epoch 4/10
20/20 [==============================] - 1s 65ms/step - loss: 0.6476 - accuracy: 0.8558 - val_loss: 0.9012 - val_accuracy: 0.7411
Epoch 5/10
20/20 [==============================] - 1s 64ms/step - loss: 0.5455 - accuracy: 0.8840 - val_loss: 0.8398 - val_accuracy: 0.7500
Epoch 6/10
20/20 [==============================] - 1s 65ms/step - loss: 0.4710 - accuracy: 0.9060 - val_loss: 0.8023 - val_accuracy: 0.7589
Epoch 7/10
20/20 [==============================] - 1s 65ms/step - loss: 0.4146 - accuracy: 0.9216 - val_loss: 0.7736 - val_accuracy: 0.7500
Epoch 8/10
2

In [11]:
efficient_net_b0_fv_main_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 keras_layer (KerasLayer)    (None, 1280)              4049564   
                                                                 
 dense (Dense)               (None, 10)                12810     
                                                                 
Total params: 4,062,374
Trainable params: 12,810
Non-trainable params: 4,049,564
_________________________________________________________________


# Fine Tuning with 10 percent of data

In [32]:
fine_tune_base_model = tf.keras.applications.EfficientNetB0(include_top=False)

fine_tune_base_model.trainable=False

In [33]:
fine_tune_base_model.summary()

Model: "efficientnetb0"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 rescaling_8 (Rescaling)        (None, None, None,   0           ['input_8[0][0]']                
                                3)                                                                
                                                                                                  
 normalization_4 (Normalization  (None, None, None,   7          ['rescaling_8[0][0]']            
 )                              3)                                                   

                                                                                                  
 block2a_se_excite (Multiply)   (None, None, None,   0           ['block2a_activation[0][0]',     
                                96)                               'block2a_se_expand[0][0]']      
                                                                                                  
 block2a_project_conv (Conv2D)  (None, None, None,   2304        ['block2a_se_excite[0][0]']      
                                24)                                                               
                                                                                                  
 block2a_project_bn (BatchNorma  (None, None, None,   96         ['block2a_project_conv[0][0]']   
 lization)                      24)                                                               
                                                                                                  
 block2b_e

                                144)                              'block3a_se_expand[0][0]']      
                                                                                                  
 block3a_project_conv (Conv2D)  (None, None, None,   5760        ['block3a_se_excite[0][0]']      
                                40)                                                               
                                                                                                  
 block3a_project_bn (BatchNorma  (None, None, None,   160        ['block3a_project_conv[0][0]']   
 lization)                      40)                                                               
                                                                                                  
 block3b_expand_conv (Conv2D)   (None, None, None,   9600        ['block3a_project_bn[0][0]']     
                                240)                                                              
          

 block4a_project_conv (Conv2D)  (None, None, None,   19200       ['block4a_se_excite[0][0]']      
                                80)                                                               
                                                                                                  
 block4a_project_bn (BatchNorma  (None, None, None,   320        ['block4a_project_conv[0][0]']   
 lization)                      80)                                                               
                                                                                                  
 block4b_expand_conv (Conv2D)   (None, None, None,   38400       ['block4a_project_bn[0][0]']     
                                480)                                                              
                                                                                                  
 block4b_expand_bn (BatchNormal  (None, None, None,   1920       ['block4b_expand_conv[0][0]']    
 ization) 

                                                                                                  
 block4c_drop (Dropout)         (None, None, None,   0           ['block4c_project_bn[0][0]']     
                                80)                                                               
                                                                                                  
 block4c_add (Add)              (None, None, None,   0           ['block4c_drop[0][0]',           
                                80)                               'block4b_add[0][0]']            
                                                                                                  
 block5a_expand_conv (Conv2D)   (None, None, None,   38400       ['block4c_add[0][0]']            
                                480)                                                              
                                                                                                  
 block5a_e

                                112)                              'block5a_project_bn[0][0]']     
                                                                                                  
 block5c_expand_conv (Conv2D)   (None, None, None,   75264       ['block5b_add[0][0]']            
                                672)                                                              
                                                                                                  
 block5c_expand_bn (BatchNormal  (None, None, None,   2688       ['block5c_expand_conv[0][0]']    
 ization)                       672)                                                              
                                                                                                  
 block5c_expand_activation (Act  (None, None, None,   0          ['block5c_expand_bn[0][0]']      
 ivation)                       672)                                                              
          

 block6b_expand_conv (Conv2D)   (None, None, None,   221184      ['block6a_project_bn[0][0]']     
                                1152)                                                             
                                                                                                  
 block6b_expand_bn (BatchNormal  (None, None, None,   4608       ['block6b_expand_conv[0][0]']    
 ization)                       1152)                                                             
                                                                                                  
 block6b_expand_activation (Act  (None, None, None,   0          ['block6b_expand_bn[0][0]']      
 ivation)                       1152)                                                             
                                                                                                  
 block6b_dwconv (DepthwiseConv2  (None, None, None,   28800      ['block6b_expand_activation[0][0]
 D)       

                                                                                                  
 block6d_expand_conv (Conv2D)   (None, None, None,   221184      ['block6c_add[0][0]']            
                                1152)                                                             
                                                                                                  
 block6d_expand_bn (BatchNormal  (None, None, None,   4608       ['block6d_expand_conv[0][0]']    
 ization)                       1152)                                                             
                                                                                                  
 block6d_expand_activation (Act  (None, None, None,   0          ['block6d_expand_bn[0][0]']      
 ivation)                       1152)                                                             
                                                                                                  
 block6d_d

                                1280)                                                             
                                                                                                  
 top_activation (Activation)    (None, None, None,   0           ['top_bn[0][0]']                 
                                1280)                                                             
                                                                                                  
Total params: 4,049,571
Trainable params: 0
Non-trainable params: 4,049,571
__________________________________________________________________________________________________


In [34]:
input_layer = tf.keras.Input(shape=(224,224,3))

x = fine_tune_base_model(input_layer,training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)

output_layer = tf.keras.layers.Dense(10,activation="softmax")(x)

In [35]:
fine_tune_main_model = tf.keras.Model(input_layer,output_layer)

In [36]:
modelsave = tf.keras.callbacks.ModelCheckpoint(filepath="FineTuneSave//modelsave.ckpt",
                                               save_weights_only=True,
                                               save_freq="epoch",
                                               verbose=1)

In [37]:
fine_tune_main_model.compile(loss="categorical_crossentropy",
                             optimizer=tf.keras.optimizers.Adam(),
                             metrics=["accuracy"])

In [38]:
for i,layer in enumerate(fine_tune_main_model.layers):
    print(f"{i},  {layer.name},  {layer.trainable}")

0,  input_9,  True
1,  efficientnetb0,  False
2,  global_average_pooling2d_3,  True
3,  dense_3,  True


In [39]:
for layer_number, layer in enumerate(fine_tune_base_model.layers):
  print(layer_number, layer.name, layer.trainable)

0 input_8 False
1 rescaling_8 False
2 normalization_4 False
3 rescaling_9 False
4 stem_conv_pad False
5 stem_conv False
6 stem_bn False
7 stem_activation False
8 block1a_dwconv False
9 block1a_bn False
10 block1a_activation False
11 block1a_se_squeeze False
12 block1a_se_reshape False
13 block1a_se_reduce False
14 block1a_se_expand False
15 block1a_se_excite False
16 block1a_project_conv False
17 block1a_project_bn False
18 block2a_expand_conv False
19 block2a_expand_bn False
20 block2a_expand_activation False
21 block2a_dwconv_pad False
22 block2a_dwconv False
23 block2a_bn False
24 block2a_activation False
25 block2a_se_squeeze False
26 block2a_se_reshape False
27 block2a_se_reduce False
28 block2a_se_expand False
29 block2a_se_excite False
30 block2a_project_conv False
31 block2a_project_bn False
32 block2b_expand_conv False
33 block2b_expand_bn False
34 block2b_expand_activation False
35 block2b_dwconv False
36 block2b_bn False
37 block2b_activation False
38 block2b_se_squeeze Fals

In [47]:
len(fine_tune_base_model.trainable_variables)

0

In [40]:
fine_tune_base_model_fg = tf.keras.applications.EfficientNetB0(include_top=False)

fine_tune_base_model_fg.trainable=True

In [41]:
input_layer = tf.keras.Input(shape=(224,224,3))

x = fine_tune_base_model_fg(input_layer,training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)

output_layer = tf.keras.layers.Dense(10,activation="softmax")(x)

In [42]:
fine_tune_main_model_fg = tf.keras.Model(input_layer,output_layer)

In [43]:
fine_tune_main_model_fg.compile(loss="categorical_crossentropy",
                             optimizer=tf.keras.optimizers.Adam(),
                             metrics=["accuracy"])

In [44]:
for i,layer in enumerate(fine_tune_main_model_fg.layers):
    print(f"{i},  {layer.name},  {layer.trainable}")

0,  input_11,  True
1,  efficientnetb0,  True
2,  global_average_pooling2d_4,  True
3,  dense_4,  True


In [45]:
for layer_number, layer in enumerate(fine_tune_base_model_fg.layers):
  print(layer_number, layer.name, layer.trainable)

0 input_10 True
1 rescaling_10 True
2 normalization_5 True
3 rescaling_11 True
4 stem_conv_pad True
5 stem_conv True
6 stem_bn True
7 stem_activation True
8 block1a_dwconv True
9 block1a_bn True
10 block1a_activation True
11 block1a_se_squeeze True
12 block1a_se_reshape True
13 block1a_se_reduce True
14 block1a_se_expand True
15 block1a_se_excite True
16 block1a_project_conv True
17 block1a_project_bn True
18 block2a_expand_conv True
19 block2a_expand_bn True
20 block2a_expand_activation True
21 block2a_dwconv_pad True
22 block2a_dwconv True
23 block2a_bn True
24 block2a_activation True
25 block2a_se_squeeze True
26 block2a_se_reshape True
27 block2a_se_reduce True
28 block2a_se_expand True
29 block2a_se_excite True
30 block2a_project_conv True
31 block2a_project_bn True
32 block2b_expand_conv True
33 block2b_expand_bn True
34 block2b_expand_activation True
35 block2b_dwconv True
36 block2b_bn True
37 block2b_activation True
38 block2b_se_squeeze True
39 block2b_se_reshape True
40 bloc

In [48]:
len(fine_tune_base_model_fg.trainable_variables)

211

In [61]:
len(fine_tune_base_model_fg.trainable_weights)

211

In [52]:
fine_tune_model_fg1 = tf.keras.applications.EfficientNetB0(include_top=False)
fine_tune_model_fg1.trainable = True

input_layer1 = tf.keras.Input(shape=(224,224,3))

x = fine_tune_model_fg1(input_layer1,training=True)
x = tf.keras.layers.GlobalAveragePooling2D()(x)

output_layer1 = tf.keras.layers.Dense(10,activation="softmax")(x)

fine_tune_main_model_fg1 = tf.keras.Model(input_layer1,output_layer1)

fine_tune_main_model_fg1.compile(loss="categorical_crossentropy",
                                 optimizer=tf.keras.optimizers.Adam(),
                                 metrics=["accuracy"])

In [55]:
for i,layer in enumerate(fine_tune_main_model_fg1.layers):
    print(f"{i},  {layer.name},  {layer.trainable}")

0,  input_17,  True
1,  efficientnetb0,  True
2,  global_average_pooling2d_6,  True
3,  dense_6,  True


In [57]:
for layer_number, layer in enumerate(fine_tune_model_fg1.layers):
  print(layer_number, layer.name, layer.trainable)

0 input_16 True
1 rescaling_18 True
2 normalization_9 True
3 rescaling_19 True
4 stem_conv_pad True
5 stem_conv True
6 stem_bn True
7 stem_activation True
8 block1a_dwconv True
9 block1a_bn True
10 block1a_activation True
11 block1a_se_squeeze True
12 block1a_se_reshape True
13 block1a_se_reduce True
14 block1a_se_expand True
15 block1a_se_excite True
16 block1a_project_conv True
17 block1a_project_bn True
18 block2a_expand_conv True
19 block2a_expand_bn True
20 block2a_expand_activation True
21 block2a_dwconv_pad True
22 block2a_dwconv True
23 block2a_bn True
24 block2a_activation True
25 block2a_se_squeeze True
26 block2a_se_reshape True
27 block2a_se_reduce True
28 block2a_se_expand True
29 block2a_se_excite True
30 block2a_project_conv True
31 block2a_project_bn True
32 block2b_expand_conv True
33 block2b_expand_bn True
34 block2b_expand_activation True
35 block2b_dwconv True
36 block2b_bn True
37 block2b_activation True
38 block2b_se_squeeze True
39 block2b_se_reshape True
40 bloc

In [58]:
len(fine_tune_model_fg1.trainable_variables)

211

In [60]:
len(fine_tune_model_fg1.trainable_weights)

211

In [13]:
fine_tune_main_model.fit(train_ds_mn_224,
                         epochs=15,
                         steps_per_epoch=len(train_ds_mn_224),
                         validation_data=valid_ds_mn_224,
                         validation_steps=len(valid_ds_mn_224),
                         callbacks=[modelsave])

Epoch 1/15
20/20 [==============================] - ETA: 0s - loss: 1.8983 - accuracy: 0.4169
Epoch 1: saving model to FineTuneSave\modelsave.ckpt
20/20 [==============================] - 11s 231ms/step - loss: 1.8983 - accuracy: 0.4169 - val_loss: 1.5883 - val_accuracy: 0.5625
Epoch 2/15
19/20 [===========================>..] - ETA: 0s - loss: 1.1358 - accuracy: 0.7516
Epoch 2: saving model to FineTuneSave\modelsave.ckpt
20/20 [==============================] - 2s 105ms/step - loss: 1.1266 - accuracy: 0.7539 - val_loss: 1.1788 - val_accuracy: 0.6518
Epoch 3/15
19/20 [===========================>..] - ETA: 0s - loss: 0.8052 - accuracy: 0.8388
Epoch 3: saving model to FineTuneSave\modelsave.ckpt
20/20 [==============================] - 2s 104ms/step - loss: 0.8003 - accuracy: 0.8370 - val_loss: 1.0009 - val_accuracy: 0.7321
Epoch 4/15
19/20 [===========================>..] - ETA: 0s - loss: 0.6379 - accuracy: 0.8618
Epoch 4: saving model to FineTuneSave\modelsave.ckpt
20/20 [===========

In [12]:
for i,layer in enumerate(fine_tune_main_model.layers):
    print(f"{i},  {layer.name},  {layer.trainable}")

0,  input_2,  True
1,  efficientnetb0,  False
2,  global_average_pooling2d,  True
3,  dense,  True


In [13]:
for layer_number, layer in enumerate(fine_tune_base_model.layers):
  print(layer_number, layer.name, layer.trainable)

0 input_1 False
1 rescaling False
2 normalization False
3 rescaling_1 False
4 stem_conv_pad False
5 stem_conv False
6 stem_bn False
7 stem_activation False
8 block1a_dwconv False
9 block1a_bn False
10 block1a_activation False
11 block1a_se_squeeze False
12 block1a_se_reshape False
13 block1a_se_reduce False
14 block1a_se_expand False
15 block1a_se_excite False
16 block1a_project_conv False
17 block1a_project_bn False
18 block2a_expand_conv False
19 block2a_expand_bn False
20 block2a_expand_activation False
21 block2a_dwconv_pad False
22 block2a_dwconv False
23 block2a_bn False
24 block2a_activation False
25 block2a_se_squeeze False
26 block2a_se_reshape False
27 block2a_se_reduce False
28 block2a_se_expand False
29 block2a_se_excite False
30 block2a_project_conv False
31 block2a_project_bn False
32 block2b_expand_conv False
33 block2b_expand_bn False
34 block2b_expand_activation False
35 block2b_dwconv False
36 block2b_bn False
37 block2b_activation False
38 block2b_se_squeeze False
39

In [14]:
fine_tune_base_model.trainable=True

for layer in fine_tune_base_model.layers[:-10]:
  layer.trainable = False

In [15]:
fine_tune_main_model.compile(loss="categorical_crossentropy",
                             optimizer=tf.keras.optimizers.Adam(),
                             metrics=["accuracy"])

In [16]:
for layer_number, layer in enumerate(fine_tune_base_model.layers):
  print(layer_number, layer.name, layer.trainable)

0 input_1 False
1 rescaling False
2 normalization False
3 rescaling_1 False
4 stem_conv_pad False
5 stem_conv False
6 stem_bn False
7 stem_activation False
8 block1a_dwconv False
9 block1a_bn False
10 block1a_activation False
11 block1a_se_squeeze False
12 block1a_se_reshape False
13 block1a_se_reduce False
14 block1a_se_expand False
15 block1a_se_excite False
16 block1a_project_conv False
17 block1a_project_bn False
18 block2a_expand_conv False
19 block2a_expand_bn False
20 block2a_expand_activation False
21 block2a_dwconv_pad False
22 block2a_dwconv False
23 block2a_bn False
24 block2a_activation False
25 block2a_se_squeeze False
26 block2a_se_reshape False
27 block2a_se_reduce False
28 block2a_se_expand False
29 block2a_se_excite False
30 block2a_project_conv False
31 block2a_project_bn False
32 block2b_expand_conv False
33 block2b_expand_bn False
34 block2b_expand_activation False
35 block2b_dwconv False
36 block2b_bn False
37 block2b_activation False
38 block2b_se_squeeze False
39

In [ ]:
fine_tune_main_model.load_weights("FineTuneSave\modelsave.ckpt")

In [18]:
fine_tune_main_model.fit(train_ds_mn_224,
                         epochs=30,
                         steps_per_epoch=len(train_ds_mn_224),
                         validation_data=valid_ds_mn_224,
                         validation_steps=len(valid_ds_mn_224),
                         initial_epoch=15,
                         callbacks=[modelsave])

Epoch 17/30
20/20 [==============================] - ETA: 0s - loss: 0.2512 - accuracy: 0.9185
Epoch 17: saving model to FineTuneSave\modelsave.ckpt
20/20 [==============================] - 8s 173ms/step - loss: 0.2512 - accuracy: 0.9185 - val_loss: 0.8857 - val_accuracy: 0.7768
Epoch 18/30
20/20 [==============================] - ETA: 0s - loss: 0.0610 - accuracy: 0.9875
Epoch 18: saving model to FineTuneSave\modelsave.ckpt
20/20 [==============================] - 2s 110ms/step - loss: 0.0610 - accuracy: 0.9875 - val_loss: 0.8006 - val_accuracy: 0.7768
Epoch 19/30
20/20 [==============================] - ETA: 0s - loss: 0.0244 - accuracy: 0.9953
Epoch 19: saving model to FineTuneSave\modelsave.ckpt
20/20 [==============================] - 2s 111ms/step - loss: 0.0244 - accuracy: 0.9953 - val_loss: 0.8891 - val_accuracy: 0.7500
Epoch 20/30
20/20 [==============================] - ETA: 0s - loss: 0.0287 - accuracy: 0.9922
Epoch 20: saving model to FineTuneSave\modelsave.ckpt
20/20 [====

KeyboardInterrupt: 